# I. Introduction <p>
Songs have become essential to many people’s lives. This project used the existing data set to train and test and give the prediction of the next popular song. Which song could become a smash hit in the future based on the dataset. The goal of this project is to use a machine learning model and predict the popularity/ hit chances of a song.
</p>

# II. Project descriptions 

<h3 style = "font-weight: bold">Data Collection</h3>
<p>- Dataset is collected from many sources on the world wide web (kaggle). Then the library (pandas, numpy, sklearn, etc) will be applied to the dataset to predict the future hits. 
</p>



These are the key attributes from dataset
- **Track**: Name of the song
- **Artist**: Name of the song writer
- **Uri**: destination source of the songs
- **Danceability**: this attribute is comparable to the track used for dancing purposes based on the combination of tempo, duration_ms, liveness, etc. Value 1.0 is the most relevant to dancing and 0.0 is lease relevant
- **Energy**: this attribute is the energy level of each track, 0.0 is the least energetic and 1.0 is most energetic. This could be the intensity of the track, loudness, tempo, speed and which instrument.
- **Key**: which key note in music is used for the song. Each integer number represents a key note and -1 value means no key is being read.
- **Loudness**: measure the moderate track level in decibels (db). The higher the value, the louder the track.
- **Mode**: ordering the notes of scale according to the interval they form with the tonic
- **Speechiness**: determine if there is spoken word in a track, if above 66 then it is likely made of spoken words, score between 0.33 and 0.66 is song that contain both music and words. Blow 0.33 means song does not have any speech
- **Acousticness**: describe from scale 0 to 1 of how acoustic a song is. If the value is 1 then the song is most likely to be an acoustic song.
- **Instrumentalness**: represent the amount of vocals in the songs, the closer it is to 1., the more instrumental the song is
- **Liveness**: refers directly to reverberation time. Live room has a long reverberation time and the dead room a short reverberation time.
- **Valence**: the musical positiveness conveyed measure from 0.0 to 1.0, track with high valence make it more positive (cheerful, happy, etc) while track with low valence sound more negative (depressed, angry, sad, etc)
- **Tempo**: the speed at which a passage of music is or should be played. This is measured from 0.0 to 1.0 where 0.0 is slowerst and 1.0 is the fastest track played.
- **Duration_ms**: length of track, the average is around three minutes and 30 seconds
- **Time_sigature**: is a notational convention used in western musical notation to specify how many beats are contained in each measure and which note is equivalent to a beat
- **Chorus_hit**: repeated section that contains the primary musical and lyrical motifs of the song
- **Sections**: is a complete but not independent, musical idea
- **Target**: is whether the song is a hit or not, 0 indicates the song is not a hit and 1 indicates that song is gonna be a hit

<h3 style = "font-weight: bold">Exploratory data analysis</h3>

- First of all, before we process data a few columns will be dropped because some of the columns will have nothing to do with our analysis (track name, artist name, and uri. There are too many elements in these columns we would have to set up too many dummy variables thus making our table with too many columns

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
warnings.filterwarnings(action='ignore')

Read the dataset using read_csv

In [4]:

df60s = pd.read_csv("dataset-of-60s.csv")
df70s = pd.read_csv("dataset-of-70s.csv")
df80s = pd.read_csv("dataset-of-80s.csv")
df90s = pd.read_csv("dataset-of-90s.csv")
df00s = pd.read_csv("dataset-of-00s.csv")
df10s = pd.read_csv("dataset-of-10s.csv")

Read dataset and shuffle data to prevent the model from learning the order of the training.

In [5]:
dfs = [pd.read_csv(f"dataset-of-{decade}0s.csv") for decade in ['6','7','8','9','0','1']]
print(dfs[1])
for i, decade in enumerate([1960,1970,1980,1990,2000,2010]):
    dfs[i]['decade'] = pd.Series(decade, index = dfs[i].index)

print(dfs[5])

#shuffle our complete df of all decades so that they are not in order

df = pd.concat(dfs, axis = 0).sample(frac = 1.0, random_state = 1).reset_index(drop = True)
print(df)

#check na and drop object because not relevant

print(df.info())

                              track                 artist  \
0                           Porque?        Reginaldo Rossi   
1                         Free Jazz   Maria Teresa Luciani   
2                 Reject of Society                  Crass   
3                            Yeah !               The Jets   
4                  Gotta Find A Way            The Moments   
...                             ...                    ...   
7761            Born with the Blues          Buster Benton   
7762       Farther On Down The Road              Joe Simon   
7763                     Sweetheart  Engelbert Humperdinck   
7764  Until It's Time For You To Go           Neil Diamond   
7765  Try Me, I Know We Can Make It           Donna Summer   

                                       uri  danceability  energy  key  \
0     spotify:track:28KC2wl7jDv6Ms1Uc0OMpm         0.669   0.547    1   
1     spotify:track:0zagn8PNnM7pM8pz1gXOAD         0.291   0.300    1   
2     spotify:track:18eNASEiuAhO2ML5

Train data using the ratio 80-20. 80% of the shuffle data will be used to train model and the 20% will be used for testing.

In [6]:
#make a copy for editing and preprocessing
def preprocess(df):
    dfCopy = df.copy()

    #we want to drop categorical values that have nothing to do with our analysis, track name, artist name, and uri (link from spotify api)
    #there are too many elements in these columns we would have to set up too many dummy variables thus making a df with too many cols
    dfCopy = dfCopy.drop(["track", "artist", "uri"], axis = 1)

    #since we predict target (hit or not) we split it

    y = dfCopy["target"]
    x = dfCopy.drop("target", axis = 1)

    #training and testing
    #higher training % = more accuracy, common practice is to use 70/30
    # due to size of our dataset (small) we will use 80/20
    x_train,  x_test, y_train, y_test = train_test_split(x,y, train_size = 0.8, shuffle = True, random_state = 1)

    #scale values to make them closer together

    scale = StandardScaler()
    scale.fit(x_train)
    x_train = pd.DataFrame(scale.transform(x_train), index = x_train.index, columns = x_train.columns)
    x_test = pd.DataFrame(scale.transform(x_test), index = x_test.index, columns = x_test.columns)

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = preprocess(df)


print(x_train.var()) #var close to 1
print(x_train.mean()) #mean close to 0

danceability        1.00003
energy              1.00003
key                 1.00003
loudness            1.00003
mode                1.00003
speechiness         1.00003
acousticness        1.00003
instrumentalness    1.00003
liveness            1.00003
valence             1.00003
tempo               1.00003
duration_ms         1.00003
time_signature      1.00003
chorus_hit          1.00003
sections            1.00003
decade              1.00003
dtype: float64
danceability       -1.148847e-16
energy              4.370127e-17
key                -6.195965e-17
loudness            2.718770e-16
mode               -5.140572e-17
speechiness        -1.174911e-17
acousticness       -6.244582e-17
instrumentalness   -1.898764e-17
liveness            1.151277e-16
valence             2.288780e-16
tempo              -1.030140e-16
duration_ms        -1.180853e-16
time_signature     -5.437270e-16
chorus_hit         -3.889359e-18
sections            1.620566e-16
decade             -5.310002e-15
dtype: fl

Train Data and individual models


In [7]:
#training data

mlModels = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier()
}


#training individual models
for name, model in mlModels.items():
    model.fit(x_train, y_train)
    print(name + " trained.")

#results

for name, model in mlModels.items():
    print(name + ": {:.2f}%".format(model.score(x_test, y_test) * 100))


                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                   Logistic Regression: 74.13%
                   K-Nearest Neighbors: 75.27%
                         Decision Tree: 72.60%
Support Vector Machine (Linear Kernel): 74.17%
   Support Vector Machine (RBF Kernel): 80.15%
                        Neural Network: 79.74%


In [17]:
dfCopy = df.copy()
dfCopy = dfCopy.drop(["track" , "artist", "uri"], axis = 1)

iris = sns.load_dataset(dfCopy)
g = sns.pairplot(iris)
plt.show()

ValueError: 'dataset-of-00s.csv' is not one of the example datasets.

In [ ]:

lr = LinearRegression